In [2]:
target = "data/bay/0001.jpeg"

In [12]:
from imageio import imread
import numpy as np

In [4]:
img = imread(target)

In [5]:
img.shape

(386, 687, 3)

In [6]:
tile_w = int(img.shape[0]/5)
tile_h = int(img.shape[1]/5)

In [7]:
tile_w, tile_h

(77, 137)

In [44]:
tile = img[0:tile_w, 0:tile_h, :].astype(int)

In [45]:
tile.shape

(77, 137, 3)

In [46]:
r_1_moment = np.mean(tile[:, :, 0])

In [47]:
r_1_moment

94.88397004455399

In [48]:
np.sum(tile[:, :, 0])/(tile.shape[0]*tile.shape[1])

94.88397004455399

In [60]:
import math

In [62]:
np.sum((tile[:, :, 0]-r_1_moment)**2)/(tile.shape[0]*tile.shape[1])

678.78547533742858

In [56]:
np.mean((tile[:, :, 0]-r_1_moment)**2)

678.78547533742858

In [66]:
r_2_moment = np.mean((tile[:, :, 0]-r_1_moment)**2)**(1/2)

In [72]:
r_2_moment

26.053511765929532

In [73]:
math.sqrt(np.mean((tile[:, :, 0]-r_1_moment)**2))

26.05351176592953

In [81]:
r_3_moment = (np.mean((tile[:, :, 0]-r_1_moment)**3))**(1./3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [68]:
r_3_moment

nan

In [91]:
np.mean((tile[:, :, 0]-r_1_moment)**3).astype(float)

-15511.299389127838

In [85]:
type(np.mean((tile[:, :, 0]-r_1_moment)**3))

numpy.float64

In [82]:
r_3_moment = np.power(np.mean((tile[:, :, 0]-r_1_moment)**3), 1/3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in power
  """Entry point for launching an IPython kernel.


In [88]:
r_3_moment = np.power(np.mean((tile[:, :, 0]-r_1_moment)**3).astype(float), 1/3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in power
  """Entry point for launching an IPython kernel.


In [89]:
r_3_moment

nan

In [105]:
-15511.299389127838**(1/3)

-24.93921198669789

In [100]:
(-15511.299389127838)**(1/3)

(12.46960599334895+21.597991130845752j)

In [95]:
np.mean((tile[:, :, 0]-r_1_moment)**3).astype(float)

-15511.299389127838

In [97]:
(np.mean((tile[:, :, 0]-r_1_moment)**3).astype(float))**(1/3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


nan

Negative cuberoot return complex answer (it also valid from the methematical point of view, but not what people want)

- https://stackoverflow.com/questions/1361740/cubic-root-of-the-negative-number-on-python
- https://stackoverflow.com/questions/45906383/python-exponent-operator-with-fractional-exponents

In [110]:
def calc_moment3(arr, moment1):
    cubed = np.mean((tile[:, :, 0]-moment1)**3)
    return math.pow(abs(cubed),1/3) * (1,-1)[cubed<0]

In [107]:
calc_moment3(tile[:, :, 0], r_1_moment)

-24.93921198669789

In [108]:
r_3_moment = moment3(tile[:, :, 0], r_1_moment)

In [111]:
def calc_moments(arr):
    moment1 = np.mean(arr)
    moment2 = np.mean((arr-moment1)**2)**(1/2)
    moment3 = calc_moment3(arr, moment1)
    return moment1, moment2, moment3

In [112]:
calc_moments(tile[:, :, 0])

(94.88397004455399, 26.053511765929532, -24.93921198669789)

In [109]:
r_1_moment, r_2_moment, r_3_moment

(94.88397004455399, 26.053511765929532, -24.93921198669789)

In [113]:
calc_moments(tile[:, :, 1])

(155.12854298985687, 17.769411514261186, -70.92930157152836)

In [114]:
calc_moments(tile[:, :, 2])

(208.25793914115081, 13.569771557507638, -119.4337022824908)

In [32]:
tmp = np.array([3, 4, 5])

In [33]:
tmp**2

array([ 9, 16, 25])

In [34]:
tmp**3

array([ 27,  64, 125])

In [39]:
itile = tile.astype(int)

In [43]:
tile[0:3, 0:3, 0]

Image([[65, 65, 65],
       [65, 65, 65],
       [68, 68, 68]], dtype=uint8)

In [40]:
itile[0:3, 0:3, 0]

Image([[65, 65, 65],
       [65, 65, 65],
       [68, 68, 68]])

In [41]:
itile[0:3, 0:3, 0]**2

Image([[4225, 4225, 4225],
       [4225, 4225, 4225],
       [4624, 4624, 4624]])

In [42]:
itile[0:3, 0:3, 0]**3

Image([[274625, 274625, 274625],
       [274625, 274625, 274625],
       [314432, 314432, 314432]])